In [4]:
!docker-compose up -d --build

ERROR: 
        Can't find a suitable configuration file in this directory or any
        parent. Are you in the right directory?

        Supported filenames: docker-compose.yml, docker-compose.yaml, compose.yml, compose.yaml
        


In [72]:
import time
from typing import Callable
from uuid import uuid4

import psycopg2

In [73]:
PG_HOST = "127.0.0.1"
# PG_HOST = "postgres"
PG_PORT = 5432
PG_DB = "postgres"
PG_USER="app"
PG_PASSWORD="123qwe"
PG_COLLECTION_LIKE = "likedFilms"
PG_COLLECTION_REVIEW = "reviews"
PG_COLLECTION_BOOKMARK = "bookmarks"
ITERATIONS_NUMBER = 10
USERS_IN_BATCH = 10
OPTIMAL_BATCH_SIZE = 200
TEST_RECORDS_SIZE = 10000

conn = psycopg2.connect(
    dbname=PG_DB,
    user=PG_USER,
    password=PG_PASSWORD,
    host=PG_HOST,
    port=PG_PORT
)
cur = conn.cursor()

In [74]:
def create_tables() -> None:
    """Создание таблиц в базе данных"""
    cur.execute("""
        CREATE TABLE IF NOT EXISTS likedFilms (
            id SERIAL PRIMARY KEY,
            user_id VARCHAR(255),
            film_id VARCHAR(255),
            type VARCHAR(255),
            datetime TIMESTAMP
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS reviews (
            id SERIAL PRIMARY KEY,
            user_id VARCHAR(255),
            film_id VARCHAR(255),
            text TEXT,
            rating INTEGER,
            datetime TIMESTAMP
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS bookmarks (
            id SERIAL PRIMARY KEY,
            user_id VARCHAR(255),
            film_id VARCHAR(255),
            datetime TIMESTAMP
        )
    """)

    conn.commit()

# Вызов функции для создания таблиц
create_tables()

In [75]:
"""Генерация фейковых данных для исследования"""
from random import choice, randint
from typing import Callable
from uuid import uuid4

from faker import Faker

LIKE = 1
DISLIKE = 0
START_DATE = "-30d"
END_DATE = "now"
MIN_RATING = 1
MAX_RATING = 10
fake = Faker()

In [76]:
def fake_like_event(user_id: str = None, film_id: str = None) -> dict:
    """Генерация события like"""
    return {
        "user_id": user_id if user_id else str(uuid4()),
        "film_id": film_id if film_id else str(uuid4()),
        "type": choice([LIKE, DISLIKE]),
        "datetime": fake.date_time_between(start_date=START_DATE, end_date=END_DATE),
    }

In [104]:
def fake_review_event(user_id: str = None, film_id: str = None) -> dict:
    """Генерация события review"""
    return {
        "user_id": user_id if user_id else str(uuid4()),
        "film_id": film_id if film_id else str(uuid4()),
        "text": fake.text(),
        "rating": randint(MIN_RATING, MAX_RATING),
        "datetime": fake.date_time_between(start_date=START_DATE, end_date=END_DATE),
    }


In [127]:
def fake_bookmark_event(user_id: str = None, film_id: str = None) -> dict:
    """Генерация события bookmark"""
    return {
        "user_id": user_id if user_id else str(uuid4()),
        "film_id": film_id if film_id else str(uuid4()),
        "datetime": fake.date_time_between(start_date=START_DATE, end_date=END_DATE),
    }

In [80]:
def fake_batch(event_faker: Callable, user_size: int, batch_size: int) -> list:
    """Генерация батча событий"""
    users = [str(uuid4()) for _ in range(user_size)]
    return [event_faker(user_id=choice(users)) for _ in range(batch_size)]

In [92]:
def fake_users_batch(event_faker: Callable, users: list, batch_size: int) -> list:
    """Генерация батча событий с фиксированными юзерами"""
    return [event_faker(user_id=choice(users)) for _ in range(batch_size)]

In [129]:
def test_insert_step(
        faker: Callable,
        table_name: str,
        batch_size: int,
        iterations: int = ITERATIONS_NUMBER,
) -> None:
    """Тестирование вставки"""
    statistics = []
    
    if table_name == PG_COLLECTION_LIKE:
        for _ in range(iterations):
            batch = fake_batch(faker, USERS_IN_BATCH, batch_size)
            query = f"INSERT INTO {PG_COLLECTION_LIKE} (user_id, film_id, type, datetime) VALUES (%s, %s, %s, %s)"
            query = query
            start = time.time()
            cur.executemany(query, [(event['user_id'], event['film_id'], event['type'], event['datetime']) for event in batch])
            conn.commit()
            end = time.time()
            statistics.append(end - start)
    
    elif table_name == PG_COLLECTION_REVIEW:
        for _ in range(iterations):
            batch = fake_batch(faker, USERS_IN_BATCH, batch_size)
            query = query = f"INSERT INTO {PG_COLLECTION_REVIEW} (user_id, film_id, text, rating, datetime) VALUES (%s, %s, %s, %s, %s)"
            query = query
            start = time.time()
            cur.executemany(query, [(event['user_id'], event['film_id'], event['text'], event['rating'], event['datetime']) for event in batch])
            conn.commit()
            end = time.time()
            statistics.append(end - start)
    
    elif table_name == PG_COLLECTION_REVIEW:
        for _ in range(iterations):
            batch = fake_batch(faker, USERS_IN_BATCH, batch_size)
            query = query = f"INSERT INTO {PG_COLLECTION_REVIEW} (user_id, film_id, text, rating, datetime) VALUES (%s, %s, %s, %s, %s)"
            query = query
            start = time.time()
            cur.executemany(query, [(event['user_id'], event['film_id'], event['text'], event['rating'], event['datetime']) for event in batch])
            conn.commit()
            end = time.time()
            statistics.append(end - start)
    
    elif table_name == PG_COLLECTION_REVIEW:
        for _ in range(iterations):
            batch = fake_batch(faker, USERS_IN_BATCH, batch_size)
            query = f"INSERT INTO {PG_COLLECTION_BOOKMARK} (user_id, film_id, datetime) VALUES (%s, %s, %s)"
            query = query
            start = time.time()
            cur.executemany(query, [(event['user_id'], event['film_id'], event['datetime']) for event in batch])
            conn.commit()
            end = time.time()
            statistics.append(end - start)
    
    if len(statistics) != 0:
        mean_batch = sum(statistics) / len(statistics)
    else:
        mean_batch = sum(statistics) / 1  # 
        
    print(
        f"Statistics for {table_name} batch_size={batch_size}: batch={mean_batch} sec, "
        f"item={mean_batch/batch_size} sec.",
    )

In [121]:
def test_insert(faker: Callable, table_name: str) -> None:
    """Тестирование вставки с разным размером батча"""
    batch_sizes = [1, 10, 50, 100, 200, 500, 1000, 2000, 5000]
    for batch_size in batch_sizes:
        test_insert_step(faker, table_name, batch_size)

In [122]:
test_insert(
    fake_like_event,
    PG_COLLECTION_LIKE,
)

Statistics for likedFilms batch_size=1: batch=0.003411221504211426 sec, item=0.003411221504211426 sec.
Statistics for likedFilms batch_size=10: batch=0.008451271057128906 sec, item=0.0008451271057128906 sec.
Statistics for likedFilms batch_size=50: batch=0.008076286315917969 sec, item=0.0001615257263183594 sec.
Statistics for likedFilms batch_size=100: batch=0.01442093849182129 sec, item=0.0001442093849182129 sec.
Statistics for likedFilms batch_size=200: batch=0.02544083595275879 sec, item=0.00012720417976379395 sec.
Statistics for likedFilms batch_size=500: batch=0.059745287895202635 sec, item=0.00011949057579040527 sec.
Statistics for likedFilms batch_size=1000: batch=0.12597131729125977 sec, item=0.00012597131729125976 sec.
Statistics for likedFilms batch_size=2000: batch=0.2575524806976318 sec, item=0.0001287762403488159 sec.
Statistics for likedFilms batch_size=5000: batch=0.5480046033859253 sec, item=0.00010960092067718505 sec.


In [123]:
test_insert(
    fake_review_event,
    PG_COLLECTION_REVIEW,
)

Statistics for reviews batch_size=1: batch=0.001703929901123047 sec, item=0.001703929901123047 sec.
Statistics for reviews batch_size=10: batch=0.0029884815216064454 sec, item=0.00029884815216064454 sec.
Statistics for reviews batch_size=50: batch=0.007425427436828613 sec, item=0.00014850854873657226 sec.
Statistics for reviews batch_size=100: batch=0.014075589179992676 sec, item=0.00014075589179992677 sec.
Statistics for reviews batch_size=200: batch=0.028784298896789552 sec, item=0.00014392149448394777 sec.
Statistics for reviews batch_size=500: batch=0.0707939624786377 sec, item=0.0001415879249572754 sec.
Statistics for reviews batch_size=1000: batch=0.11413779258728027 sec, item=0.00011413779258728027 sec.
Statistics for reviews batch_size=2000: batch=0.22256641387939452 sec, item=0.00011128320693969726 sec.
Statistics for reviews batch_size=5000: batch=0.5512250900268555 sec, item=0.00011024501800537109 sec.


In [130]:
test_insert(
    fake_bookmark_event,
    PG_COLLECTION_BOOKMARK,
)

Statistics for bookmarks batch_size=1: batch=0.0 sec, item=0.0 sec.
Statistics for bookmarks batch_size=10: batch=0.0 sec, item=0.0 sec.
Statistics for bookmarks batch_size=50: batch=0.0 sec, item=0.0 sec.
Statistics for bookmarks batch_size=100: batch=0.0 sec, item=0.0 sec.
Statistics for bookmarks batch_size=200: batch=0.0 sec, item=0.0 sec.
Statistics for bookmarks batch_size=500: batch=0.0 sec, item=0.0 sec.
Statistics for bookmarks batch_size=1000: batch=0.0 sec, item=0.0 sec.
Statistics for bookmarks batch_size=2000: batch=0.0 sec, item=0.0 sec.
Statistics for bookmarks batch_size=5000: batch=0.0 sec, item=0.0 sec.


In [152]:
def test_read_data(faker: Callable, table_name: str, users_size: int) -> None:
    """Тестирование чтения"""
    statistics = []
    users = [str(uuid4()) for _ in range(users_size)]
    
    for user in users:
        start = time.time()
        query = f"SELECT * FROM {table_name} WHERE user_id = %s"
        cur.execute(query, (user,))
        _ = cur.fetchall()
        statistics.append(time.time() - start)

    mean_batch = sum(statistics) / len(statistics)
    print(
        f"Statistics read for {table_name} for ~{int(TEST_RECORDS_SIZE/users_size)} records: {mean_batch} sec",
    )

In [153]:
test_read_data(fake_like_event, PG_COLLECTION_LIKE, 20)

Statistics read for likedFilms for ~500 records: 0.06465929746627808 sec


In [154]:
test_read_data(fake_review_event, PG_COLLECTION_REVIEW, 20)

Statistics read for reviews for ~500 records: 0.016708147525787354 sec


In [155]:
test_read_data(fake_review_event, PG_COLLECTION_REVIEW, 20)

Statistics read for reviews for ~500 records: 0.01640894412994385 sec
